In [1]:
import numpy as np
# and IPython.display for audio output
import IPython.display

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display
import time

In [11]:
audio_path = "../genres/hiphop/hiphop.00000.au"
y, sr = librosa.load(audio_path)

In [14]:
# S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
# We have to truncate the data for valid layer dimensions



y_harmonic, y_percussive = librosa.effects.hpss(y)
S_harmonic = librosa.feature.melspectrogram(y_harmonic, sr=sr)[:,:TRUNC_VAL]
S_percussive = librosa.feature.melspectrogram(y_percussive, sr=sr)[:,:TRUNC_VAL]

print(f"Harmonic shape: {y_harmonic.shape}")
print(f"Spectogram Harmonic shape: {S_harmonic.shape}")
print(f"stacked shape: {np.stack([S_harmonic, S_percussive], axis=-1).shape}")

Harmonic shape: (661504,)
Spectogram Harmonic shape: (128, 1288)
stacked shape: (128, 1288, 2)


In [4]:
SPECT_TRUNC_VAL = 1288
TRUNC = 661504

genres = ["hiphop","blues","classical","country","disco","jazz","metal","pop","reggae","rock"]
num_classes = len(genres)
stacks_y = {0:[], 1:[], 2:[]}
stacks_separates = []
y_list = {0:[], 1:[], 2:[]}
# 0 training -- 1 validation -- 2 test
raw_y_s = []
start = time.time()
for i, genre in enumerate(genres):
    for k in range(100):
        path = "../genres/%s/%s.%s.au" % (genre, genre, str(k).zfill(5))
        print(path)
        index = 1 if k%10 == 0 else 0
        test_index = 2 if k%20 == 0 else 0
        index = max(index, test_index)
        y, sr = librosa.load(path)
#         raw_y_s.append(y[:TRUNC])
        stacks_y[index].append(librosa.feature.melspectrogram(y, sr=sr)[:,:SPECT_TRUNC_VAL])
        y_harmonic, y_percussive = librosa.effects.hpss(y[:TRUNC])
        S_harmonic_i = librosa.feature.melspectrogram(y_harmonic, sr=sr)[:,:SPECT_TRUNC_VAL]
        S_percussive_i = librosa.feature.melspectrogram(y_percussive, sr=sr)[:,:SPECT_TRUNC_VAL]
        stacks_separates.append(np.stack([S_harmonic_i, S_percussive_i], axis=-1))
        y_list[index].append(i)
end = time.time()
print(end - start)
x = np.stack(stacks_y[0], axis=0)
x = np.expand_dims(x, axis=-1)
x_val = np.stack(stacks_y[1], axis=0)
x_val = np.expand_dims(x_val, axis=-1)
x_test = np.stack(stacks_y[2], axis=0)
x_test = np.expand_dims(x_test, axis=-1)
# x_separates = np.stack(stacks_separates, axis=0)
y = np.array(y_list[0])
y_val = np.array(y_list[1])
y_test = np.array(y_list[2])
# raw_y = np.stack(raw_y_s, axis=0)
# raw_y.shape

../genres/hiphop/hiphop.00000.au
../genres/hiphop/hiphop.00001.au
../genres/hiphop/hiphop.00002.au
../genres/hiphop/hiphop.00003.au
../genres/hiphop/hiphop.00004.au
../genres/hiphop/hiphop.00005.au
../genres/hiphop/hiphop.00006.au
../genres/hiphop/hiphop.00007.au
../genres/hiphop/hiphop.00008.au
../genres/hiphop/hiphop.00009.au
../genres/hiphop/hiphop.00010.au
../genres/hiphop/hiphop.00011.au
../genres/hiphop/hiphop.00012.au
../genres/hiphop/hiphop.00013.au
../genres/hiphop/hiphop.00014.au
../genres/hiphop/hiphop.00015.au
../genres/hiphop/hiphop.00016.au
../genres/hiphop/hiphop.00017.au
../genres/hiphop/hiphop.00018.au
../genres/hiphop/hiphop.00019.au
../genres/hiphop/hiphop.00020.au
../genres/hiphop/hiphop.00021.au
../genres/hiphop/hiphop.00022.au
../genres/hiphop/hiphop.00023.au
../genres/hiphop/hiphop.00024.au
../genres/hiphop/hiphop.00025.au
../genres/hiphop/hiphop.00026.au
../genres/hiphop/hiphop.00027.au
../genres/hiphop/hiphop.00028.au
../genres/hiphop/hiphop.00029.au
../genres/

../genres/classical/classical.00046.au
../genres/classical/classical.00047.au
../genres/classical/classical.00048.au
../genres/classical/classical.00049.au
../genres/classical/classical.00050.au
../genres/classical/classical.00051.au
../genres/classical/classical.00052.au
../genres/classical/classical.00053.au
../genres/classical/classical.00054.au
../genres/classical/classical.00055.au
../genres/classical/classical.00056.au
../genres/classical/classical.00057.au
../genres/classical/classical.00058.au
../genres/classical/classical.00059.au
../genres/classical/classical.00060.au
../genres/classical/classical.00061.au
../genres/classical/classical.00062.au
../genres/classical/classical.00063.au
../genres/classical/classical.00064.au
../genres/classical/classical.00065.au
../genres/classical/classical.00066.au
../genres/classical/classical.00067.au
../genres/classical/classical.00068.au
../genres/classical/classical.00069.au
../genres/classical/classical.00070.au
../genres/classical/class

../genres/disco/disco.00084.au
../genres/disco/disco.00085.au
../genres/disco/disco.00086.au
../genres/disco/disco.00087.au
../genres/disco/disco.00088.au
../genres/disco/disco.00089.au
../genres/disco/disco.00090.au
../genres/disco/disco.00091.au
../genres/disco/disco.00092.au
../genres/disco/disco.00093.au
../genres/disco/disco.00094.au
../genres/disco/disco.00095.au
../genres/disco/disco.00096.au
../genres/disco/disco.00097.au
../genres/disco/disco.00098.au
../genres/disco/disco.00099.au
../genres/jazz/jazz.00000.au
../genres/jazz/jazz.00001.au
../genres/jazz/jazz.00002.au
../genres/jazz/jazz.00003.au
../genres/jazz/jazz.00004.au
../genres/jazz/jazz.00005.au
../genres/jazz/jazz.00006.au
../genres/jazz/jazz.00007.au
../genres/jazz/jazz.00008.au
../genres/jazz/jazz.00009.au
../genres/jazz/jazz.00010.au
../genres/jazz/jazz.00011.au
../genres/jazz/jazz.00012.au
../genres/jazz/jazz.00013.au
../genres/jazz/jazz.00014.au
../genres/jazz/jazz.00015.au
../genres/jazz/jazz.00016.au
../genres/j

../genres/pop/pop.00063.au
../genres/pop/pop.00064.au
../genres/pop/pop.00065.au
../genres/pop/pop.00066.au
../genres/pop/pop.00067.au
../genres/pop/pop.00068.au
../genres/pop/pop.00069.au
../genres/pop/pop.00070.au
../genres/pop/pop.00071.au
../genres/pop/pop.00072.au
../genres/pop/pop.00073.au
../genres/pop/pop.00074.au
../genres/pop/pop.00075.au
../genres/pop/pop.00076.au
../genres/pop/pop.00077.au
../genres/pop/pop.00078.au
../genres/pop/pop.00079.au
../genres/pop/pop.00080.au
../genres/pop/pop.00081.au
../genres/pop/pop.00082.au
../genres/pop/pop.00083.au
../genres/pop/pop.00084.au
../genres/pop/pop.00085.au
../genres/pop/pop.00086.au
../genres/pop/pop.00087.au
../genres/pop/pop.00088.au
../genres/pop/pop.00089.au
../genres/pop/pop.00090.au
../genres/pop/pop.00091.au
../genres/pop/pop.00092.au
../genres/pop/pop.00093.au
../genres/pop/pop.00094.au
../genres/pop/pop.00095.au
../genres/pop/pop.00096.au
../genres/pop/pop.00097.au
../genres/pop/pop.00098.au
../genres/pop/pop.00099.au
.

In [12]:
SHORT_LEN = int(np.floor(1288/3))

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Conv3D, MaxPooling2D, Conv2D, Flatten, BatchNormalization, Dropout
from keras.utils import to_categorical
from keras.callbacks import LambdaCallback
from keras.optimizers import Adam
from keras.backend import gradients, eval, get_value
from keras import backend as K
print(x.shape)
# print(x_separates.shape)
M, H, W, C = x.shape
# random_test = np.random.randn(M, H, W, C)

(900, 128, 1288, 1)


In [6]:
def normalize_data(data, mu=None, std=None):
    if mu is None or std is None:
        mu = np.mean(data)
        std = np.std(data)
        
    norm_data = data.copy() - mu
    norm_data /= std
    return norm_data, mu, std

In [38]:
import tensorflow as tf
sess = tf.Session()

K.set_session(sess)


model = Sequential()
model.add(Conv2D(
    filters=16,
    kernel_size=5,
    data_format="channels_last",
    input_shape=(128,SPECT_TRUNC_VAL,1)
))
model.add(BatchNormalization())
model.add(MaxPooling2D(padding="valid"))
model.add(Conv2D(
    filters=32,
    kernel_size=5
))
model.add(BatchNormalization())
model.add(MaxPooling2D(padding="valid"))
model.add(Conv2D(
    filters=64,
    kernel_size=5
))
# model.add(MaxPooling2D(padding="valid"))
model.add(Flatten())
# model.add(Dense(
#     units=10,
#     activation='relu'
# ))
model.add(Dense(
    units=10,
    activation='softmax'
))
lr = .1
lr1 = .01
lr2 = .05  # .550
lr3 = .001 # .850
lr4 = .0005 # .9
lr5 = .0001 # 1
lr6 = .00005
# lr6 = .00005 # 
model.compile(optimizer=Adam(lr=lr6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [249]:
# print_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[2].get_weights()[0][0,0,0]))
print_gradients = LambdaCallback(on_epoch_end=lambda batch, logs: print(eval(gradients(model.layers[7].output, model.trainable_weights)[0].eval())))
print_last_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[len(model.layers)-1].get_weights()[0][0]))
print_first_weights = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[0].get_weights()[0][0]))
print_softmax = LambdaCallback(on_epoch_end=lambda batch, logs: print(model.layers[8].output.eval(session=sess)))


In [39]:
norm_x, mu, std = normalize_data(x)
norm_x_val, mu, std = normalize_data(x_val, mu, std)

In [40]:
y_cat = to_categorical(y, num_classes=10)
y_val_cat = to_categorical(y_val, num_classes=10)
# print(y_cat)
model.fit(x,y_cat,batch_size=10, epochs=10, validation_data=(x_val, y_val_cat))

Train on 900 samples, validate on 50 samples
Epoch 1/10
900/900 [==============================] - 208s 231ms/step - loss: 2.6562 - acc: 0.3189 - val_loss: 1.7297 - val_acc: 0.3800
Epoch 2/10
900/900 [==============================] - 204s 227ms/step - loss: 0.5516 - acc: 0.8644 - val_loss: 1.6863 - val_acc: 0.4200
Epoch 3/10
900/900 [==============================] - 199s 221ms/step - loss: 0.3387 - acc: 0.9356 - val_loss: 1.6905 - val_acc: 0.4400
Epoch 4/10
900/900 [==============================] - 203s 225ms/step - loss: 0.2495 - acc: 0.9600 - val_loss: 1.7376 - val_acc: 0.4200
Epoch 5/10
900/900 [==============================] - 194s 215ms/step - loss: 0.1888 - acc: 0.9678 - val_loss: 1.9617 - val_acc: 0.4400
Epoch 6/10
900/900 [==============================] - 193s 215ms/step - loss: 0.1453 - acc: 0.9822 - val_loss: 1.8938 - val_acc: 0.4400
Epoch 7/10
900/900 [==============================] - 185s 206ms/step - loss: 0.1212 - acc: 0.9878 - val_loss: 1.9314 - val_acc: 0.4800
Epo

In [7]:
def make_short_x(x):
    M, H, W, C = x.shape
    x_short = np.zeros((M*3, H, SHORT_LEN, 1))
    for idx, data in enumerate(x):
        for i in range(3):
            x_short[idx] = data[:,i*SHORT_LEN:(i+1)*SHORT_LEN,:]
    return x_short

def make_short_y(y):
    M = y.shape
    y_short = np.zeros(M[0]*3)
    print(y_short.shape)
    for idx, val in enumerate(y):
        y_short[idx] = val
        y_short[idx+1] = val
        y_short[idx+2] = val
    return y_short
    

In [13]:
x_short = make_short_x(x)
x_val_short = make_short_x(x_val)
x_test_short = make_short_x(x_test)
y_short = make_short_y(y)
y_val_short = make_short_y(y_val)
y_test_short = make_short_y(y_test)

(2700,)
(150,)
(150,)


In [14]:
import tensorflow as tf
sess = tf.Session()

K.set_session(sess)


model_short = Sequential()
model_short.add(Conv2D(
    filters=16,
    kernel_size=5,
    data_format="channels_last",
    input_shape=(128,SHORT_LEN,1)
))
model_short.add(BatchNormalization())
model_short.add(MaxPooling2D(padding="valid"))
model_short.add(Conv2D(
    filters=32,
    kernel_size=5
))
model_short.add(BatchNormalization())
model_short.add(MaxPooling2D(padding="valid"))
model_short.add(Conv2D(
    filters=64,
    kernel_size=5
))
# model.add(MaxPooling2D(padding="valid"))
model_short.add(Flatten())
# model.add(Dense(
#     units=10,
#     activation='relu'
# ))
model_short.add(Dense(
    units=10,
    activation='softmax'
))
lr = .1
lr1 = .01
lr2 = .05  # .550
lr3 = .001 # .850
lr4 = .0005 # .9
lr5 = .0001 # 1
lr6 = .00005
# lr6 = .00005 # 
model_short.compile(optimizer=Adam(lr=lr6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
y_cat_short = to_categorical(y_short, num_classes=10)
y_val_short_cat = to_categorical(y_val_short, num_classes=10)
# print(y_cat)
model_short.fit(x_short,y_cat_short,batch_size=10, epochs=10, validation_data=(x_val_short, y_val_short_cat))

Train on 2700 samples, validate on 150 samples
Epoch 1/10
2700/2700 [==============================] - 14s 5ms/step - loss: 1.3087 - acc: 0.7215 - val_loss: 1.1632 - val_acc: 0.7000
Epoch 2/10
2700/2700 [==============================] - 11s 4ms/step - loss: 0.5751 - acc: 0.8707 - val_loss: 1.1238 - val_acc: 0.7200
Epoch 3/10
2700/2700 [==============================] - 11s 4ms/step - loss: 0.3930 - acc: 0.9200 - val_loss: 1.1179 - val_acc: 0.7067
Epoch 4/10
2700/2700 [==============================] - 11s 4ms/step - loss: 0.3199 - acc: 0.9304 - val_loss: 1.2575 - val_acc: 0.7333
Epoch 5/10
2700/2700 [==============================] - 11s 4ms/step - loss: 0.2319 - acc: 0.9496 - val_loss: 1.2779 - val_acc: 0.7400
Epoch 6/10
2700/2700 [==============================] - 11s 4ms/step - loss: 0.1949 - acc: 0.9526 - val_loss: 1.3311 - val_acc: 0.7333
Epoch 7/10
2700/2700 [==============================] - 11s 4ms/step - loss: 0.1776 - acc: 0.9589 - val_loss: 1.3623 - val_acc: 0.7533
Epoch 8/

In [46]:
import tensorflow as tf
sess = tf.Session()

K.set_session(sess)


model_short_2 = Sequential()
model_short_2.add(Conv2D(
    filters=16,
    kernel_size=5,
    data_format="channels_last",
    input_shape=(128,SHORT_LEN,1)
))
model_short_2.add(BatchNormalization())
model_short_2.add(MaxPooling2D(padding="valid"))
model_short_2.add(Conv2D(
    filters=32,
    kernel_size=5
))
model_short_2.add(BatchNormalization())
model_short_2.add(MaxPooling2D(padding="valid"))
model_short_2.add(Conv2D(
    filters=64,
    kernel_size=5
))
model_short_2.add(MaxPooling2D(padding="valid"))
model_short_2.add(Flatten())
model_short_2.add(Dropout(0.5))
model_short_2.add(Dense(
    units=100,
    activation='tanh'
))
model_short_2.add(Dense(
    units=10,
    activation='softmax'
))
lr = .1
lr1 = .01
lr2 = .05  # .550
lr3 = .001 # .850
lr4 = .0005 # .9
lr5 = .0001 # 1
lr6 = .00005
# lr6 = .00005 # 
model_short_2.compile(optimizer=Adam(lr=lr6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
model_short_2.fit(x_short,y_cat_short,batch_size=20, epochs=20, validation_data=(x_val_short, y_val_short_cat))

Train on 2700 samples, validate on 150 samples
Epoch 1/20
2700/2700 [==============================] - 10s 4ms/step - loss: 1.2893 - acc: 0.7007 - val_loss: 1.1800 - val_acc: 0.7333
Epoch 2/20
2700/2700 [==============================] - 9s 3ms/step - loss: 1.1483 - acc: 0.7093 - val_loss: 1.1184 - val_acc: 0.7067
Epoch 3/20
2700/2700 [==============================] - 9s 3ms/step - loss: 1.0983 - acc: 0.7215 - val_loss: 1.0914 - val_acc: 0.7133
Epoch 4/20
2700/2700 [==============================] - 9s 3ms/step - loss: 1.0625 - acc: 0.7185 - val_loss: 1.0713 - val_acc: 0.7200
Epoch 5/20
2700/2700 [==============================] - 9s 3ms/step - loss: 1.0301 - acc: 0.7281 - val_loss: 1.0328 - val_acc: 0.7133
Epoch 6/20
2700/2700 [==============================] - 9s 3ms/step - loss: 0.9954 - acc: 0.7304 - val_loss: 1.0196 - val_acc: 0.7133
Epoch 7/20
2700/2700 [==============================] - 9s 3ms/step - loss: 0.9597 - acc: 0.7378 - val_loss: 1.0150 - val_acc: 0.7200ETA: 0s - loss:

In [27]:
norm_x_short, mu, std = normalize_data(x_short)
norm_x_val_short, mu, std = normalize_data(x_val_short, mu, std)
norm_x_test_short, mu, std = normalize_data(x_test_short, mu, std)

In [47]:
model_short_2.fit(norm_x_short,y_cat_short,batch_size=40, epochs=40, validation_data=(norm_x_val_short, y_val_short_cat))

Train on 2700 samples, validate on 150 samples
Epoch 1/40
2700/2700 [==============================] - 10s 4ms/step - loss: 1.0252 - acc: 0.6930 - val_loss: 0.8984 - val_acc: 0.7067
Epoch 2/40
2700/2700 [==============================] - 8s 3ms/step - loss: 0.7917 - acc: 0.7326 - val_loss: 0.8510 - val_acc: 0.7133
Epoch 3/40
2700/2700 [==============================] - 8s 3ms/step - loss: 0.6828 - acc: 0.7748 - val_loss: 0.7946 - val_acc: 0.7333
Epoch 4/40
2700/2700 [==============================] - 8s 3ms/step - loss: 0.6153 - acc: 0.7970 - val_loss: 0.8004 - val_acc: 0.7333
Epoch 5/40
2700/2700 [==============================] - 8s 3ms/step - loss: 0.5693 - acc: 0.8122 - val_loss: 0.7612 - val_acc: 0.7467
Epoch 6/40
2700/2700 [==============================] - 8s 3ms/step - loss: 0.5186 - acc: 0.8341 - val_loss: 0.7393 - val_acc: 0.7600
Epoch 7/40
2700/2700 [==============================] - 8s 3ms/step - loss: 0.4810 - acc: 0.8622 - val_loss: 0.7374 - val_acc: 0.7667
Epoch 8/40
270

In [52]:
predictions = model_short_2.predict(x,verbose=1)

20/20 [==============================] - 2s 109ms/step


In [53]:
for i in range(20):
    pred = np.argmax(predictions[i])
    print(f"{pred}: {pred == np.argmax(y_cat[i])}")

0: True
0: True
0: True
0: True
0: True
0: True
0: True
0: True
0: True
0: True
0: False
1: True
1: True
0: False
1: True
1: True
1: True
1: True
1: True
1: True


In [217]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_71 (Conv2D)           (None, 124, 1284, 16)     416       
_________________________________________________________________
batch_normalization_3 (Batch (None, 124, 1284, 16)     64        
_________________________________________________________________
max_pooling2d_52 (MaxPooling (None, 62, 642, 16)       0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 58, 638, 32)       12832     
_________________________________________________________________
batch_normalization_4 (Batch (None, 58, 638, 32)       128       
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 29, 319, 32)       0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 25, 315, 64)       51264     
__________

In [ ]:
model.get_weights()

In [233]:
predictions_test = model.predict(random_test,verbose=1, callbacks=[print_softmax])
for i in range(len(predictions_test)):
    print(predictions_test)

In [253]:
model_fc = Sequential()
model_fc.add(Dense(
    units=100,
    input_shape=(661504,)
))
model_fc.add(Dense(
    units=10,
    activation='softmax'
))
# model_fc.summary()
model_fc.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [254]:
model_fc.fit(raw_y,y_cat,batch_size=1, epochs=3, callbacks=[print_first_weights])

Epoch 1/3
4/4 [==============================] - 9s 2s/step - loss: 8.5786 - acc: 0.5000
[[[-0.00141054  0.03924088  0.00982541 -0.06723718  0.14631334
    0.085247    0.05593383 -0.00029854  0.19803205  0.10322731
    0.19210014  0.12397878  0.05366305 -0.155586   -0.16645752
    0.15057439]]

 [[-0.18966101  0.16679089  0.16780674  0.08289624  0.16095227
   -0.02773692  0.00476591 -0.05361988 -0.08788291  0.11735255
   -0.01341543 -0.13236004  0.06324315 -0.18394592 -0.1097433
   -0.00178529]]

 [[-0.06578009 -0.01824477 -0.05911817  0.19440772 -0.01845913
    0.02624493 -0.16905391 -0.17659797 -0.02613381  0.02919289
    0.12753558 -0.00067317  0.13613641 -0.01125458 -0.11592694
   -0.0182863 ]]

 [[-0.04725617 -0.2088394  -0.09113347 -0.12641612  0.10209908
    0.18108234 -0.213449    0.02864265  0.00988443  0.07360809
    0.01936161 -0.18102069  0.20523158 -0.18567625 -0.01277931
   -0.10194732]]

 [[-0.15339404  0.01524298 -0.18633261 -0.0212782  -0.03636252
   -0.02153432 -0.001